## Klasifikátory kuchýň 2

V tejto druhej lekcii o klasifikácii sa pozrieme na `ďalšie spôsoby`, ako klasifikovať kategóriálne údaje. Tiež sa oboznámime s dôsledkami výberu jedného klasifikátora oproti inému.

### [**Kvíz pred prednáškou**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/23/)

### **Predpoklady**

Predpokladáme, že ste dokončili predchádzajúce lekcie, pretože budeme nadväzovať na niektoré koncepty, ktoré sme sa už naučili.

Na túto lekciu budeme potrebovať nasledujúce balíky:

-   `tidyverse`: [tidyverse](https://www.tidyverse.org/) je [kolekcia balíkov pre R](https://www.tidyverse.org/packages), ktorá robí dátovú vedu rýchlejšou, jednoduchšou a zábavnejšou!

-   `tidymodels`: [tidymodels](https://www.tidymodels.org/) je [rámec balíkov](https://www.tidymodels.org/packages/) určený na modelovanie a strojové učenie.

-   `themis`: [balík themis](https://themis.tidymodels.org/) poskytuje dodatočné kroky pre recepty na riešenie nevyvážených údajov.

Môžete ich nainštalovať pomocou:

`install.packages(c("tidyverse", "tidymodels", "kernlab", "themis", "ranger", "xgboost", "kknn"))`

Prípadne, nasledujúci skript skontroluje, či máte nainštalované potrebné balíky na dokončenie tohto modulu, a v prípade, že chýbajú, ich nainštaluje za vás.


In [ ]:
suppressWarnings(if (!require("pacman"))install.packages("pacman"))

pacman::p_load(tidyverse, tidymodels, themis, kernlab, ranger, xgboost, kknn)

## **1. Mapa klasifikácie**

V našej [predchádzajúcej lekcii](https://github.com/microsoft/ML-For-Beginners/tree/main/4-Classification/2-Classifiers-1) sme sa snažili odpovedať na otázku: ako si vybrať medzi viacerými modelmi? Do veľkej miery to závisí od charakteristík údajov a typu problému, ktorý chceme vyriešiť (napríklad klasifikácia alebo regresia?).

Predtým sme sa naučili o rôznych možnostiach, ktoré máte pri klasifikácii údajov, pomocou prehľadovej tabuľky od Microsoftu. Pythonovský rámec pre strojové učenie, Scikit-learn, ponúka podobnú, ale podrobnejšiu prehľadovú tabuľku, ktorá vám môže ďalej pomôcť zúžiť výber vašich odhadovačov (iný výraz pre klasifikátory):

<p >
   <img src="../../images/map.png"
   width="700"/>
   <figcaption></figcaption>


> Tip: [navštívte túto mapu online](https://scikit-learn.org/stable/tutorial/machine_learning_map/) a kliknite na cestu, aby ste si prečítali dokumentáciu.
>
> [Referenčná stránka Tidymodels](https://www.tidymodels.org/find/parsnip/#models) tiež poskytuje vynikajúcu dokumentáciu o rôznych typoch modelov.

### **Plán** 🗺️

Táto mapa je veľmi užitočná, ak máte jasné pochopenie svojich údajov, pretože sa môžete „prejsť“ po jej cestách k rozhodnutiu:

-   Máme viac ako 50 vzoriek

-   Chceme predpovedať kategóriu

-   Máme označené údaje

-   Máme menej ako 100K vzoriek

-   ✨ Môžeme si vybrať Linear SVC

-   Ak to nefunguje, keďže máme numerické údaje

    -   Môžeme skúsiť ✨ KNeighbors Classifier

        -   Ak to nefunguje, skúste ✨ SVC a ✨ Ensemble Classifiers

Toto je veľmi užitočná cesta, ktorú sa oplatí nasledovať. Teraz sa do toho pustíme pomocou [tidymodels](https://www.tidymodels.org/) frameworku na modelovanie: konzistentnej a flexibilnej kolekcie balíkov v R, vyvinutej na podporu správnej štatistickej praxe 😊.

## 2. Rozdelenie údajov a riešenie nevyváženého dátového súboru.

Z našich predchádzajúcich lekcií sme sa naučili, že existuje súbor spoločných ingrediencií naprieč našimi kuchyňami. Tiež sme si všimli, že rozdelenie počtu kuchýň bolo dosť nerovnomerné.

S týmto sa vysporiadame takto:

-   Odstránime najbežnejšie ingrediencie, ktoré spôsobujú zmätok medzi odlišnými kuchyňami, pomocou `dplyr::select()`.

-   Použijeme `recipe`, ktorý predspracuje údaje, aby boli pripravené na modelovanie, aplikovaním algoritmu `over-sampling`.

Toto sme už prešli v predchádzajúcej lekcii, takže to bude hračka 🥳!


In [ ]:
# Load the core Tidyverse and Tidymodels packages
library(tidyverse)
library(tidymodels)

# Load the original cuisines data
df <- read_csv(file = "https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/4-Classification/data/cuisines.csv")

# Drop id column, rice, garlic and ginger from our original data set
df_select <- df %>% 
  select(-c(1, rice, garlic, ginger)) %>%
  # Encode cuisine column as categorical
  mutate(cuisine = factor(cuisine))


# Create data split specification
set.seed(2056)
cuisines_split <- initial_split(data = df_select,
                                strata = cuisine,
                                prop = 0.7)

# Extract the data in each split
cuisines_train <- training(cuisines_split)
cuisines_test <- testing(cuisines_split)

# Display distribution of cuisines in the training set
cuisines_train %>% 
  count(cuisine) %>% 
  arrange(desc(n))

### Riešenie nevyvážených údajov

Nevyvážené údaje často negatívne ovplyvňujú výkon modelu. Mnohé modely dosahujú najlepšie výsledky, keď je počet pozorovaní rovnaký, a preto majú tendenciu zápasiť s nevyváženými údajmi.

Existujú dva hlavné spôsoby, ako riešiť nevyvážené dátové súbory:

-   pridanie pozorovaní do minoritnej triedy: `Over-sampling`, napríklad pomocou algoritmu SMOTE, ktorý synteticky generuje nové príklady minoritnej triedy na základe najbližších susedov týchto prípadov.

-   odstránenie pozorovaní z majoritnej triedy: `Under-sampling`

V našej predchádzajúcej lekcii sme ukázali, ako riešiť nevyvážené dátové súbory pomocou `recipe`. Recipe si môžete predstaviť ako plán, ktorý popisuje, aké kroky by sa mali aplikovať na dátový súbor, aby bol pripravený na analýzu údajov. V našom prípade chceme dosiahnuť rovnomerné rozdelenie počtu našich kuchýň pre náš `training set`. Poďme na to.


In [ ]:
# Load themis package for dealing with imbalanced data
library(themis)

# Create a recipe for preprocessing training data
cuisines_recipe <- recipe(cuisine ~ ., data = cuisines_train) %>%
  step_smote(cuisine) 

# Print recipe
cuisines_recipe

Teraz sme pripravení trénovať modely 👩‍💻👨‍💻!

## 3. Nad rámec multinomických regresných modelov

V našej predchádzajúcej lekcii sme sa zaoberali multinomickými regresnými modelmi. Poďme preskúmať niektoré flexibilnejšie modely pre klasifikáciu.

### Support Vector Machines.

V kontexte klasifikácie je `Support Vector Machines` technika strojového učenia, ktorá sa snaží nájsť *hyperrovinu*, ktorá "najlepšie" oddeľuje triedy. Pozrime sa na jednoduchý príklad:

<p >
   <img src="../../images/svm.png"
   width="300"/>
   <figcaption>https://commons.wikimedia.org/w/index.php?curid=22877598</figcaption>


H1~ neoddeľuje triedy. H2~ ich oddeľuje, ale iba s malým okrajom. H3~ ich oddeľuje s maximálnym okrajom.

#### Lineárny Support Vector Classifier

Support-Vector clustering (SVC) je súčasťou rodiny techník strojového učenia Support-Vector machines. V SVC je hyperplocha vybraná tak, aby správne oddelila `väčšinu` tréningových pozorovaní, ale `môže nesprávne klasifikovať` niektoré pozorovania. Tým, že umožníme niektorým bodom byť na nesprávnej strane, SVM sa stáva odolnejším voči odľahlým hodnotám, a tým lepšie generalizuje na nové dáta. Parameter, ktorý reguluje toto porušenie, sa nazýva `cost` a má predvolenú hodnotu 1 (pozri `help("svm_poly")`).

Vytvorme lineárny SVC nastavením `degree = 1` v polynomiálnom SVM modeli.


In [ ]:
# Make a linear SVC specification
svc_linear_spec <- svm_poly(degree = 1) %>% 
  set_engine("kernlab") %>% 
  set_mode("classification")

# Bundle specification and recipe into a worklow
svc_linear_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(svc_linear_spec)

# Print out workflow
svc_linear_wf

Teraz, keď sme zachytili kroky predspracovania a špecifikáciu modelu do *workflow*, môžeme pokračovať a natrénovať lineárny SVC a zároveň vyhodnotiť výsledky. Pre metriky výkonu vytvorme súbor metrík, ktorý bude hodnotiť: `presnosť`, `citlivosť`, `pozitívnu prediktívnu hodnotu` a `F mieru`.

> `augment()` pridá stĺpec/stĺpce s predikciami k daným údajom.


In [ ]:
# Train a linear SVC model
svc_linear_fit <- svc_linear_wf %>% 
  fit(data = cuisines_train)

# Create a metric set
eval_metrics <- metric_set(ppv, sens, accuracy, f_meas)


# Make predictions and Evaluate model performance
svc_linear_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

#### Support Vector Machine

Support vector machine (SVM) je rozšírením klasifikátora podporných vektorov, ktoré umožňuje zohľadniť nelineárnu hranicu medzi triedami. V podstate SVM využívajú *kernel trick* na rozšírenie priestoru vlastností, aby sa prispôsobili nelineárnym vzťahom medzi triedami. Jednou z populárnych a mimoriadne flexibilných funkcií jadra, ktorú SVM využívajú, je *Radial basis function.* Pozrime sa, ako bude fungovať na našich údajoch.


In [ ]:
set.seed(2056)

# Make an RBF SVM specification
svm_rbf_spec <- svm_rbf() %>% 
  set_engine("kernlab") %>% 
  set_mode("classification")

# Bundle specification and recipe into a worklow
svm_rbf_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(svm_rbf_spec)


# Train an RBF model
svm_rbf_fit <- svm_rbf_wf %>% 
  fit(data = cuisines_train)


# Make predictions and Evaluate model performance
svm_rbf_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

Oveľa lepšie 🤩!

> ✅ Pozrite si:
>
> -   [*Support Vector Machines*](https://bradleyboehmke.github.io/HOML/svm.html), Hands-on Machine Learning with R
>
> -   [*Support Vector Machines*](https://www.statlearning.com/), An Introduction to Statistical Learning with Applications in R
>
> pre ďalšie čítanie.

### Klasifikátory najbližšieho suseda

Algoritmus *K*-najbližšieho suseda (KNN) predpovedá každé pozorovanie na základe jeho *podobnosti* s ostatnými pozorovaniami.

Poďme ho aplikovať na naše dáta.


In [ ]:
# Make a KNN specification
knn_spec <- nearest_neighbor() %>% 
  set_engine("kknn") %>% 
  set_mode("classification")

# Bundle recipe and model specification into a workflow
knn_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(knn_spec)

# Train a boosted tree model
knn_wf_fit <- knn_wf %>% 
  fit(data = cuisines_train)


# Make predictions and Evaluate model performance
knn_wf_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

Zdá sa, že tento model nefunguje až tak dobre. Pravdepodobne zmena argumentov modelu (pozrite si `help("nearest_neighbor")`) zlepší jeho výkon. Určite to vyskúšajte.

> ✅ Pozrite si:
>
> -   [Hands-on Machine Learning with R](https://bradleyboehmke.github.io/HOML/)
>
> -   [An Introduction to Statistical Learning with Applications in R](https://www.statlearning.com/)
>
> aby ste sa dozvedeli viac o klasifikátoroch *K*-Najbližších Susedov.

### Ensemble klasifikátory

Ensemble algoritmy fungujú tak, že kombinujú viacero základných odhadov, aby vytvorili optimálny model, buď:

`bagging`: aplikáciou *priemerovacej funkcie* na kolekciu základných modelov

`boosting`: vytváraním sekvencie modelov, ktoré na seba nadväzujú, aby zlepšili prediktívny výkon.

Začnime tým, že vyskúšame model Random Forest, ktorý vytvára veľkú kolekciu rozhodovacích stromov a potom aplikuje priemerovaciu funkciu na vytvorenie lepšieho celkového modelu.


In [ ]:
# Make a random forest specification
rf_spec <- rand_forest() %>% 
  set_engine("ranger") %>% 
  set_mode("classification")

# Bundle recipe and model specification into a workflow
rf_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(rf_spec)

# Train a random forest model
rf_wf_fit <- rf_wf %>% 
  fit(data = cuisines_train)


# Make predictions and Evaluate model performance
rf_wf_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

Dobrá práca 👏!

Poďme tiež experimentovať s modelom Boosted Tree.

Boosted Tree definuje ensemble metódu, ktorá vytvára sériu sekvenčných rozhodovacích stromov, kde každý strom závisí od výsledkov predchádzajúcich stromov v snahe postupne znižovať chybu. Zameriava sa na váhy nesprávne klasifikovaných položiek a upravuje prispôsobenie pre ďalší klasifikátor, aby ich opravil.

Existujú rôzne spôsoby, ako tento model prispôsobiť (pozri `help("boost_tree")`). V tomto príklade prispôsobíme Boosted stromy pomocou enginu `xgboost`.


In [ ]:
# Make a boosted tree specification
boost_spec <- boost_tree(trees = 200) %>% 
  set_engine("xgboost") %>% 
  set_mode("classification")

# Bundle recipe and model specification into a workflow
boost_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(boost_spec)

# Train a boosted tree model
boost_wf_fit <- boost_wf %>% 
  fit(data = cuisines_train)


# Make predictions and Evaluate model performance
boost_wf_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

> ✅ Pozrite si:
>
> -   [Machine Learning for Social Scientists](https://cimentadaj.github.io/ml_socsci/tree-based-methods.html#random-forests)
>
> -   [Hands-on Machine Learning with R](https://bradleyboehmke.github.io/HOML/)
>
> -   [An Introduction to Statistical Learning with Applications in R](https://www.statlearning.com/)
>
> -   <https://algotech.netlify.app/blog/xgboost/> - Skúma model AdaBoost, ktorý je dobrou alternatívou k xgboost.
>
> pre viac informácií o Ensemble klasifikátoroch.

## 4. Extra - porovnanie viacerých modelov

V tomto cvičení sme vytvorili pomerne veľké množstvo modelov 🙌. Môže byť únavné alebo náročné vytvárať množstvo workflowov z rôznych sád predspracovania a/alebo špecifikácií modelov a potom jeden po druhom vypočítavať metriky výkonnosti.

Pozrime sa, či to môžeme vyriešiť vytvorením funkcie, ktorá aplikuje zoznam workflowov na tréningovú množinu a následne vráti metriky výkonnosti na základe testovacej množiny. Použijeme `map()` a `map_dfr()` z balíka [purrr](https://purrr.tidyverse.org/) na aplikáciu funkcií na každý prvok zoznamu.

> Funkcie [`map()`](https://purrr.tidyverse.org/reference/map.html) vám umožňujú nahradiť mnoho for-cyklov kódom, ktorý je stručnejší a ľahšie čitateľný. Najlepším miestom na učenie sa o funkciách [`map()`](https://purrr.tidyverse.org/reference/map.html) je [kapitola o iterácii](http://r4ds.had.co.nz/iteration.html) v knihe R for Data Science.


In [ ]:
set.seed(2056)

# Create a metric set
eval_metrics <- metric_set(ppv, sens, accuracy, f_meas)

# Define a function that returns performance metrics
compare_models <- function(workflow_list, train_set, test_set){
  
  suppressWarnings(
    # Fit each model to the train_set
    map(workflow_list, fit, data = train_set) %>% 
    # Make predictions on the test set
      map_dfr(augment, new_data = test_set, .id = "model") %>%
    # Select desired columns
      select(model, cuisine, .pred_class) %>% 
    # Evaluate model performance
      group_by(model) %>% 
      eval_metrics(truth = cuisine, estimate = .pred_class) %>% 
      ungroup()
  )
  
} # End of function

In [ ]:
# Make a list of workflows
workflow_list <- list(
  "svc" = svc_linear_wf,
  "svm" = svm_rbf_wf,
  "knn" = knn_wf,
  "random_forest" = rf_wf,
  "xgboost" = boost_wf)

# Call the function
set.seed(2056)
perf_metrics <- compare_models(workflow_list = workflow_list, train_set = cuisines_train, test_set = cuisines_test)

# Print out performance metrics
perf_metrics %>% 
  group_by(.metric) %>% 
  arrange(desc(.estimate)) %>% 
  slice_head(n=7)

# Compare accuracy
perf_metrics %>% 
  filter(.metric == "accuracy") %>% 
  arrange(desc(.estimate))


[**workflowset**](https://workflowsets.tidymodels.org/) balík umožňuje používateľom vytvárať a jednoducho prispôsobovať veľké množstvo modelov, ale je primárne navrhnutý na prácu s technikami resamplingu, ako je `krížová validácia`, ktorú si ešte len preberieme.

## **🚀Výzva**

Každá z týchto techník má veľké množstvo parametrov, ktoré môžete upravovať, napríklad `cost` v SVM, `neighbors` v KNN, `mtry` (náhodne vybrané prediktory) v Random Forest.

Preskúmajte predvolené parametre každého z nich a zamyslite sa nad tým, čo by úprava týchto parametrov znamenala pre kvalitu modelu.

Ak chcete zistiť viac o konkrétnom modeli a jeho parametroch, použite: `help("model")`, napr. `help("rand_forest")`.

> V praxi zvyčajne *odhadujeme* *najlepšie hodnoty* týchto parametrov trénovaním mnohých modelov na `simulovanom dátovom súbore` a meraním, ako dobre tieto modely fungujú. Tento proces sa nazýva **ladenie**.

### [**Kvíz po prednáške**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/24/)

### **Prehľad a samostatné štúdium**

V týchto lekciách je veľa odborných výrazov, preto si nájdite chvíľu na preštudovanie [tohto zoznamu](https://docs.microsoft.com/dotnet/machine-learning/resources/glossary?WT.mc_id=academic-77952-leestott) užitočnej terminológie!

#### POĎAKOVANIE:

[`Allison Horst`](https://twitter.com/allison_horst/) za vytvorenie úžasných ilustrácií, ktoré robia R prístupnejším a pútavejším. Viac ilustrácií nájdete v jej [galérii](https://www.google.com/url?q=https://github.com/allisonhorst/stats-illustrations&sa=D&source=editors&ust=1626380772530000&usg=AOvVaw3zcfyCizFQZpkSLzxiiQEM).

[Cassie Breviu](https://www.twitter.com/cassieview) a [Jen Looper](https://www.twitter.com/jenlooper) za vytvorenie pôvodnej verzie tohto modulu v Pythone ♥️

Šťastné učenie,

[Eric](https://twitter.com/ericntay), Zlatý študentský ambasádor Microsoft Learn.

<p >
   <img src="../../images/r_learners_sm.jpeg"
   width="569"/>
   <figcaption>Ilustrácia od @allison_horst</figcaption>



---

**Upozornenie**:  
Tento dokument bol preložený pomocou služby na automatický preklad [Co-op Translator](https://github.com/Azure/co-op-translator). Aj keď sa snažíme o presnosť, upozorňujeme, že automatické preklady môžu obsahovať chyby alebo nepresnosti. Pôvodný dokument v jeho pôvodnom jazyku by mal byť považovaný za autoritatívny zdroj. Pre dôležité informácie sa odporúča profesionálny ľudský preklad. Nezodpovedáme za akékoľvek nedorozumenia alebo nesprávne interpretácie vyplývajúce z použitia tohto prekladu.
